In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

# os.chdir("drive/My Drive/PCNN/")
os.chdir("drive/My Drive/Big Data Project/")

In [ ]:
!unzip PCNN

Archive:  PCNN.zip
   creating: PCNN/
   creating: PCNN/data_cache/
  inflating: PCNN/run.py             
  inflating: PCNN/plot.py            
  inflating: PCNN/predict_output.json  
  inflating: PCNN/.DS_Store          
  inflating: PCNN/config.py          
  inflating: PCNN/LICENSE            
  inflating: PCNN/preprocess.py      
  inflating: PCNN/pr_pcnn.png        
  inflating: PCNN/script.sh          
  inflating: PCNN/Untitled.ipynb     
  inflating: PCNN/pr_cnn.png         
  inflating: PCNN/predict_data.pkl   
  inflating: PCNN/selector.py        
   creating: PCNN/output/
   creating: PCNN/__pycache__/
  inflating: PCNN/encoder.py         
  inflating: PCNN/model.py           
  inflating: PCNN/README.md          
  inflating: PCNN/utils.py           
  inflating: __MACOSX/PCNN/._utils.py  
  inflating: PCNN/evaluate.py        
   creating: PCNN/.ipynb_checkpoints/
  inflating: PCNN/train.log          
  inflating: __MACOSX/PCNN/._train.log  
   creating: PCNN/processed/
   

In [ ]:
os.chdir("PCNN/")
!ls
!python3 run.py

config.py   evaluate.py   plot.py	       processed    script.sh
data	    LICENSE	  pr_cnn.png	       pr_pcnn.png  selector.py
data_cache  model.py	  predict_data.pkl     __pycache__  train.log
draw.py     new_data.txt  predict_output.json  README.md    Untitled.ipynb
encoder.py  output	  preprocess.py        run.py	    utils.py
--------------------------------------
some config:
data_dir = ./processed
output_dir = ./output
cache_dir = ./data_cache
embedding_path = ./processed/word2vec_50d.txt
embedding_dim = 50
min_freq = 0
encoder = pcnn
selector = att
model_name = PCNN+ATT
mode = 0
seed = 2020
cuda = 0
epoch = 5
batch_size = 160
dropout = 0.5
lr = 0.001
max_len = 120
pos_dis = 45
pos_dim = 5
filter_num = 230
window = 3
device = cuda:0
model_dir = ./output/PCNN+ATT
--------------------------------------
start to load data ...
embedding scale: 91345*50d
finish loading embeddng!
tcmalloc: large alloc 2248876032 bytes == 0x558af0936000 @  0x7fa172e8db6b 0x7fa172ead379 0x7fa0724c0cde 0x7f

In [ ]:
import json

# json.load(open("dataset/aylien_covid_news_data.jsonl"))

In [ ]:
!pip install spacy
!pip install scispacy
# !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_md-0.4.0.tar.gz
!python -m spacy download en_core_web_sm

# !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_sm-0.2.4.tar.gz

     |████████████████████████████████| 13.7 MB 5.3 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
# !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz

     |████████████████████████████████| 125.1 MB 48 kB/s 
  Created wheel for en-ner-bc5cdr-md: filename=en_ner_bc5cdr_md-0.4.0-py3-none-any.whl size=125666882 sha256=4cd68189d9baeee7aa0bee95afb02f99a903d4b714926a2192a9042b8e9913b1
  Stored in directory: /root/.cache/pip/wheels/6a/17/04/ed8f50ffdcb9b9e4ce601c0c3cf88361f2b3f54a10c0fa0803
Successfully built en-ner-bc5cdr-md


In [ ]:
# import scispacy
# import spacy
# import en_ner_bc5cdr_md   #The model we are going to use
# from spacy import displacy
# from scispacy.abbreviation import AbbreviationDetector
# from scispacy.umls_linking import UmlsEntityLinker
import spacy
from spacy import displacy
from collections import Counter
# import en_core_web_sm
# people”, “business”, “person”, and “location”. 
# Place-Place, Person-Place, Language-Place,
# Person, GPE, LOC, ORG, 
import json

In [ ]:
nlp = spacy.load("en_core_web_sm")

def get_sent(text):
    doc = nlp(text)
    return list(doc.sents)

def ner(text):
    # nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    # print(o)
    entities = [[X.text, X.start] for X in doc.ents if X.label_ in ['PERSON', 'GPE', 'LOC', 'ORG', 'NORP', 'FAC']]
        
    
    return entities

In [ ]:
count = 0
import string
fw = open("./final_predict.json", 'w', encoding='utf-8')

with open('./dataset/aylien_covid_news_data.jsonl', 'r') as json_file:

    for json_str in json_file:
      result = json.loads(json_str)
      sents = get_sent(result['body'])

      for sent in sents:
        text = str(sent)
        text = text.replace('\t', ' ')
        text = text.replace('\n', ' ')
        text = text.translate(text.maketrans('','',string.punctuation))
        text = text.replace(u'\xa0',' ')
        text = text.strip()
        text = " ".join(text.split())
        
        entities = ner(text)

        for i in range (len(entities)):
            if " " in entities[i][0]:
                new = entities[i][0].replace(" ", "_")
                text = text.replace(entities[i][0], new)
                entities[i][0] = new


        pairs = [[a[0], b[0]] for idx, a in enumerate(entities) for b in entities[idx + 1:] if len(entities) >= 2 and a[1] < b[1] and a[0] != b[0]]
      
        for pair in pairs: 
              instance = dict(
                      head=pair[0],
                      tail=pair[1],
                      sentence=text,
                  )

              # pri .nt(instance)
              print(instance)
              json.dump(instance, fw, ensure_ascii=False)
              fw.write('\n')
      # print(list(doc.sents))
      # for x in doc.ents:
      #   print(x.label_)

      # entity = doc.ents[0]
      # print(entity.ent_type_)
      # for umls_ent in entity._.umls_ents:
      #   print(linker.umls.cui_to_entity[umls_ent[0]])

      count += 1
      print(count)
      if count == 10000:
        break
      

Streaming output truncated to the last 5000 lines.
{'head': 'Taiwan', 'tail': 'China', 'sentence': 'Taiwan also avoided the type of strict lockdowns that characterized the response in China and many other countries'}
{'head': 'Taiwans', 'tail': 'Europe', 'sentence': 'Taiwans relative success in stymying the coronavirus outbreak has largely been overshadowed by the worsening crisis in Europe and the US and the sense that if any lessons were there to be learned the moment has passed'}
{'head': 'Taiwans', 'tail': 'US', 'sentence': 'Taiwans relative success in stymying the coronavirus outbreak has largely been overshadowed by the worsening crisis in Europe and the US and the sense that if any lessons were there to be learned the moment has passed'}
{'head': 'Europe', 'tail': 'US', 'sentence': 'Taiwans relative success in stymying the coronavirus outbreak has largely been overshadowed by the worsening crisis in Europe and the US and the sense that if any lessons were there to be learned the

In [ ]:
from spacy import displacy
displacy_image = displacy.render(sent, jupyter = True, style = 'ent')



/usr/local/lib/python3.7/dist-packages/spacy/displacy/__init__.py:189: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [ ]:
fw.close()